In [ ]:
import os
import json
import time
from cryptography.fernet import Fernet

# Generate encryption keys for each component
key_database = {
    "client": Fernet.generate_key(),
    "AS": Fernet.generate_key(),
    "TGS": Fernet.generate_key(),
    "service": Fernet.generate_key()
}

# ==========================
# 🔐 Encryption & Decryption
# ==========================

def encrypt_data(key, data):
    """Encrypts a JSON object using Fernet symmetric encryption."""
    f = Fernet(key) # Creates Fernet object with a assigned key
    data_string = json.dumps(data) # Turns data from a json to a string
    data_bytes = bytes(data_string, "utf-8") # Turns data from a string to bytes for encryption
    encrypted_data = f.encrypt(data_bytes) # Encrypts data using Fernet object
    return encrypted_data 

def decrypt_data(key, encrypted_data):
    """Decrypts a JSON object using Fernet symmetric encryption."""
    f = Fernet(key) # Creates Fernet object with a assigned key
    decrypted_data = f.decrypt(encrypted_data) # Decrypts data using Fernet object
    decrypted_data_json = json.loads(decrypted_data) # Turns Decrypted data back into a json object
    return decrypted_data_json


# ==========================
# 🎟️ Authentication Server (AS) - Issues Ticket Granting Ticket (TGT)
# ==========================

def authentication_server(client_id):
    """Simulates the Authentication Server issuing a TGT to the client."""
    session_key = Fernet.generate_key() # Generates the new session key for the client
    nonce = generate_nonce() # Generates a new random value to ensure uqnicnes
    issue_time = time.time() # Gets current time in seconds for validity test later
    ticket = { # Creates the ticket as a json object
        "client_id" : client_id,
        "session_key" : bytes.decode(session_key), # Json data must be a string session_key is in bytes, bytes.decode defaults to utf-8
        "nonce" : nonce,
        "issue_time" : issue_time
    }

    encrypted_ticket = encrypt_data(key_database["AS"], ticket) # Encrypts the ticket using the Authentication Server key

    return encrypted_ticket, session_key

# ==========================
# 🎫 Ticket Granting Server (TGS) - Issues Service Ticket
# ==========================

def ticket_granting_server(encrypted_tgt, client_session_key):
    """Simulates the Ticket Granting Server issuing a service ticket."""
    try:
        tgt = decrypt_data(key_database["AS"], encrypted_tgt) # Decrypts the tgt using the Authentication Server key
        current_time = time.time() # Gets current time in seconds to both check freshness and use in service ticket

        if current_time - tgt["issue_time"] >= 300: raise Exception("Ticket Expired") # Checks if tgt time is within 300 seconds or 5 minutes of current time
        if tgt["session_key"] != bytes.decode(client_session_key): raise Exception("Ticket session key invaild") # Ensures that tgt client and current client are the same

        service_session_key = Fernet.generate_key() # Generates new key for the new ticket
        nonce = generate_nonce() # Generates a new nonce for the new ticket to ensure uqniceness

        service_ticket = { # Creates new service ticket as a json object
            "client_id": tgt["client_id"],
            "service_session_key": bytes.decode(service_session_key), # Json data must be a string session_key is in bytes, bytes.decode defaults to utf-8
            "issued_at": current_time,
            "nonce": nonce
        }

        encrypted_service_ticket = encrypt_data(key_database["TGS"], service_ticket) # Encrypts service ticket using Ticket Granting Server key

        return encrypted_service_ticket, service_session_key

    except Exception as e:
        return f"❌ Error in TGS: {str(e)}", None

# ==========================
# 🏛️ Service Server - Grants Access
# ==========================

def service_server(encrypted_service_ticket):
    """Validates and decrypts the service ticket before granting access."""
    try:
        ticket = decrypt_data(key_database["TGS"], encrypted_service_ticket) # Decrypts service ticket using Ticket Granting Server key

        if time.time() - ticket["issued_at"] > 300: raise Exception("Service Ticket Expired") # Checks if service ticket time is within 300 seconds or 5 minutes of current time

        return f"✅ Access granted to {ticket["client_id"]} with key {ticket["service_session_key"]}" # Grantes access if nothing fails

    except Exception as e:
        return f"❌ Service Authentication Failed: {str(e)}"

# ==========================
# Nonce Generater - Generates random data
# ==========================
def generate_nonce(length=8):
    """Generates a random string of hex values of any given length"""
    random_data = os.urandom(length) # Generates random bytes and combines them up to specified length
    random_data_hex = bytes.hex(random_data) # Converts byte data into hex format
    return random_data_hex

# ==========================
# 🖥️ Interactive CLI
# ==========================

def main():
    # """Interactive command-line interface for running Kerberos authentication steps."""
    client_id = "client"
    tgt = None
    client_session_key = None
    service_ticket = None
    service_session_key = None

    while True:
        print("\n===== Kerberos Authentication System =====")
        print("1. Request Ticket Granting Ticket (TGT)")
        print("2. Request Service Ticket")
        print("3. Access Protected Service")
        print("4. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == "1":
            tgt, client_session_key = authentication_server(client_id) # Creates tgt ticket and session key for a client id

            # Resets varibles used in later choices
            service_ticket = None 
            service_session_key = None

            print(f"\n🔐 Received TGT:")
            print(tgt)
            print(f"\n🔐 Received Client Session Key:")
            print(client_session_key)


        elif choice == "2":
            service_ticket, service_session_key = ticket_granting_server(tgt, client_session_key) # Creates service ticket from a tgt ticket for a client
            print(f"\n🎟️ Received Service Ticket:")
            print(service_ticket)
            print(f"\n🎟️ Received Service Session Key:")
            print(service_session_key)

        elif choice == "3":
            print(f"\n {service_server(service_ticket)}") # Returnes if service ticket grants access or not

        elif choice == "4":
            print("\n🚪 Exiting the system.")
            break   # Closes program

        else:
            print("\n❌ Invalid choice, please enter a valid option.")

    # ==========================
    # Automatic Section Tests
    # ==========================

    # # Encrypt and Deecrypt data test
    # test_key = key_database["client"] 
    # test_data = {
    #     "test" : "Hello World",
    #     "foo" : "bar",
    #     "123" : 456
    # }
    # print(f"Input: {test_data}")
    # encrypted = encrypt_data(test_key, test_data)
    # print(f"Encrypted token: {encrypted}")
    # print(f"Decrypted data: {decrypt_data(test_key, encrypted)}")
    
    # # Authentication Server data test
    # tgt, client_sess = authentication_server("client")
    # print("Encrypted TGT:", tgt)
    # print("Client session key:", client_sess)
    # print("Decrypted TGT :", decrypt_data(key_database["AS"], tgt))

    # # Ticket Granting Server data test
    # tgt, client_sk = authentication_server("client")
    # svc_ticket, svc_sk = ticket_granting_server(tgt, client_sk)
    # print("Encrypted service ticket:", svc_ticket)
    # print("Decrypted service ticket (debug):", decrypt_data(key_database["TGS"], svc_ticket))
    # print("Service session key:", svc_sk)

    # # Service Server
    # tgt, client_sk = authentication_server("client")
    # svc_ticket = ticket_granting_server(tgt, client_sk)[0]
    # print(service_server(svc_ticket))


# Run the interactive simulation
if __name__ == "__main__":
    main()


Input: {'test': 'Hello World', 'foo': 'bar', '123': 456}
Encrypted token: b'gAAAAABpelmdTpSZ2Ml6ikspYhOH99NtpVaaD47PYB7p5D_9rR2-vor_5yHYhy05csmkaR-LW7C3xQmqLq9U6nNgcjbCUgVesR5RnI3CdEgyIWL2j9MSUqyQEGVIKlQlfk3Ija1pop76NKifCtQKoDAAx0mrtth9HQ=='
Decrypted data: {'test': 'Hello World', 'foo': 'bar', '123': 456}
